<a href="https://colab.research.google.com/github/sunshineluyao/UTXO/blob/main/Partition_the_data_by_spent_year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


run the following code in big query and generate joint_date
reference: https://cloud.google.com/bigquery/docs/reference/standard-sql/timestamp_functions
```
SELECT 
  value,
  FORMAT_TIMESTAMP("%Y-%m-%d", block_timestamp) AS block_date,
  FORMAT_TIMESTAMP("%Y-%m-%d", spent_block_timestamp) AS spent_block_date,
  EXTRACT(YEAR FROM block_timestamp) AS year,
  EXTRACT(Month FROM block_timestamp) AS Month
FROM `crypto-291811.UTXO.joint_all`
```



Create Partition Table
https://fivetran.com/docs/destinations/bigquery/partition-table

run the following to create dateset partition by the spent year, do the same for all block years until 2020


```
SELECT 
  value,
  block_date,
  spent_block_date
FROM `crypto-291811.UTXO.joint_date` 
WHERE year=2009 or year=2010
```




In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'crypto-291811'
client = bigquery.Client(project=project_id)

for dataset in client.list_datasets():
  print(dataset.dataset_id)

Now, we can first deal with the data that has creation year in 2009 or 2010, Note that the data is smaller than 1GB so that we can import directly to colab. 
After we deal with the data, we need to delete all data that has been spent up to 2010. 

In [ ]:
###To retrieve table rows as a pandas.DataFrame:
# from google.cloud import bigquery
# client = bigquery.Client()
project = "crypto-291811"
dataset_id = "UTXO"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("joint_2010")
table = client.get_table(table_ref)

df_2010 = client.list_rows(table).to_dataframe()
df_2010.head()

In [65]:
df_2010.tail()

,value,block_date,spent_block_date
266462,5000000000,2010-10-11,2020-10-11
266463,5000000000,2010-10-13,2020-10-11
266464,5000000000,2010-11-10,2020-10-11
266465,5000000000,2010-11-09,2020-10-14
266466,5000000000,2010-09-23,2020-10-17


In [ ]:
# Importing drive method from colab for accessing google drive
from google.colab import drive

In [73]:
# Mounting drive
# This will require authentication : Follow the steps as guided
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [74]:
df_2010.to_csv('/content/drive/My Drive/UTXO/joint_2010.csv')

In [75]:
###To retrieve table rows as a pandas.DataFrame:
# from google.cloud import bigquery
# client = bigquery.Client()
project = "crypto-291811"
dataset_id = "UTXO"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("joint_2011")
table = client.get_table(table_ref)

df_2011 = client.list_rows(table).to_dataframe()
df_2011.head()

,value,block_date,spent_block_date
0,5003000000,2011-03-05,2011-04-15
1,3120000000,2011-12-11,2011-12-11
2,5328081,2011-12-10,2011-12-19
3,1114760,2011-10-14,2011-10-14
4,53683000000,2011-06-03,2011-06-03


In [76]:
df_2011.to_csv('/content/drive/My Drive/UTXO/joint_2011.csv')

Let's first test the data for 2010 and 2011

In [66]:
### sort the data by block_date
df_2010.dtypes

value               object
block_date          object
spent_block_date    object
dtype: object

In [67]:
import pandas as pd

In [68]:
df_2010['block_date']=pd.to_datetime(df_2010['block_date'])
df_2010['spent_block_date']=pd.to_datetime(df_2010['spent_block_date'])

In [69]:
df_2010.dtypes

value                       object
block_date          datetime64[ns]
spent_block_date    datetime64[ns]
dtype: object

In [70]:
df_2010=df_2010.sort_values(by=['block_date','spent_block_date'])
df_2010.head()

,value,block_date,spent_block_date
0,5000000000,2009-01-03,NaT
41092,5000000000,2009-01-09,2009-01-12
1,5000000000,2009-01-09,NaT
2,5000000000,2009-01-09,NaT
3,5000000000,2009-01-09,NaT


In [71]:
df_2010.tail()

,value,block_date,spent_block_date
41087,5000000,2010-12-31,NaT
41088,5000000,2010-12-31,NaT
41089,5000000,2010-12-31,NaT
41090,5000000,2010-12-31,NaT
41091,5000000000,2010-12-31,NaT


In [77]:
df_2011['block_date']=pd.to_datetime(df_2011['block_date'])
df_2011['spent_block_date']=pd.to_datetime(df_2011['spent_block_date'])

In [80]:
df_2011=df_2011.sort_values(by=['block_date','spent_block_date'])
df_2011.head(100)

,value,block_date,spent_block_date
8317,12461000000,2011-01-01,2011-01-01
23422,12646000000,2011-01-01,2011-01-01
25177,33905000000,2011-01-01,2011-01-01
33561,12451000000,2011-01-01,2011-01-01
35431,33910000000,2011-01-01,2011-01-01
...,...,...,...
720058,12591000000,2011-01-01,2011-01-01
721292,4407000000,2011-01-01,2011-01-01
723992,4586000000,2011-01-01,2011-01-01
730701,1518000000,2011-01-01,2011-01-01


In [79]:
df_2011.tail()

,value,block_date,spent_block_date
4557104,50000000,2011-12-31,NaT
4557779,50000000,2011-12-31,NaT
4563227,50000000,2011-12-31,NaT
4621699,101000000,2011-12-31,NaT
4627254,101000000,2011-12-31,NaT
